In [1]:
import pandas as pd

In [2]:
import numpy as np

In [4]:
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split 
# from sklearn.cross_validation import KFold
from sklearn.model_selection import KFold

In [17]:
import lightgbm as lgb

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [23]:
from sklearn.metrics import accuracy_score

In [6]:
model_sample = pd.read_csv('dataset/model.csv')
model_sample.set_index('user_id',inplace=True)
label = model_sample[['y']]
model_sample = model_sample.drop('y',axis=1)

In [7]:
def get_features_middle(data):
    model_sample_strong_feature = data.copy()
    # 将身份信息以及财产信息进行编码
    first_strong_features = ['x_003','x_004','x_005','x_006','x_007','x_008','x_009','x_010','x_011','x_012','x_013','x_014','x_015','x_016','x_017','x_018','x_019']
    res = 0
    for i in range(len(first_strong_features)):
        res += 2 ** i * data[first_strong_features[i]]

    model_sample_strong_feature['x_1_strong'] = res 
    # 借记卡的比例特征
    model_sample_strong_feature['x_022/x_020'] = data['x_022'] / (data['x_020'] + 1e-10)
    model_sample_strong_feature['x_023/x_020'] = data['x_023'] / (data['x_020'] + 1e-10)
    model_sample_strong_feature['x_024/x_020'] = data['x_024'] /  (data['x_020'] + 1e-10)
    model_sample_strong_feature['x_025/x_020'] = data['x_025'] /  (data['x_020']+ 1e-10)
    model_sample_strong_feature['x_026/x_020'] = data['x_026'] /  (data['x_020'] + 1e-10)
    
    # 贷记卡的比例特征
    model_sample_strong_feature['x_028/x_021'] = data['x_028'] / (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_029/x_021'] = data['x_029'] / (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_030/x_021'] = data['x_030'] /  (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_031/x_021'] = data['x_031'] /  (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_032/x_021'] = data['x_032'] /  (data['x_021'] + 1e-10)
    
    # 银行卡的比例特征
    model_sample_strong_feature['all_cards'] = (data['x_034'] +  data['x_035'] + data['x_036'] + data['x_037'] + data['x_038'] + data['x_039'] + data['x_040']  ).values

    model_sample_strong_feature['x_034/all_cards'] = data['x_034'] / (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_035/all_cards'] = data['x_035'] / (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_036/all_cards'] = data['x_036'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_037/all_cards'] = data['x_037'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_038/all_cards'] = data['x_038'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_039/all_cards'] = data['x_039'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_040/all_cards'] = data['x_040'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
   
    # 标准差还原
    model_sample_strong_feature['x_043/x_044'] = data['x_043'] / (data['x_044'] + 1e-10)
    model_sample_strong_feature['x_046/x_047'] = data['x_046'] / (data['x_047'] + 1e-10)
    model_sample_strong_feature['x_050/x_051'] = data['x_050'] / (data['x_051'] + 1e-10)
    model_sample_strong_feature['x_053/x_054'] = data['x_053'] / (data['x_054'] + 1e-10)
    model_sample_strong_feature['x_057/x_058'] = data['x_057'] / (data['x_058'] + 1e-10)
    model_sample_strong_feature['x_060/x_061'] = data['x_060'] / (data['x_061'] + 1e-10)
    model_sample_strong_feature['x_076/x_077'] = data['x_076'] / (data['x_077'] + 1e-10)
    model_sample_strong_feature['x_079/x_080'] = data['x_079'] / (data['x_080'] + 1e-10)
    model_sample_strong_feature['x_083/x_084'] = data['x_083'] / (data['x_084'] + 1e-10)
    model_sample_strong_feature['x_086/x_087'] = data['x_086'] / (data['x_087'] + 1e-10)
    model_sample_strong_feature['x_090/x_091'] = data['x_090'] / (data['x_091'] + 1e-10)
    model_sample_strong_feature['x_094/x_095'] = data['x_094'] / (data['x_095'] + 1e-10)
    model_sample_strong_feature['x_098/x_099'] = data['x_098'] / (data['x_099'] + 1e-10)
    model_sample_strong_feature['x_123/x_124'] = data['x_123'] / (data['x_124'] + 1e-10)
    model_sample_strong_feature['x_126/x_127'] = data['x_126'] / (data['x_127'] + 1e-10)

    
    # 每张卡（信用or其他）交易金额等；每笔（异地每笔）交易金额等；每笔还款金额等；每笔商旅，保险，家装，金融等的均值特征；每个月的平均交易笔数；其他有意义的均值特征
    
    model_sample_strong_feature['x_045/x_41'] = data['x_045'] / (data['x_041'] + 1e-10)
    model_sample_strong_feature['x_052/x_48'] = data['x_052'] / (data['x_048'] + 1e-10)
    model_sample_strong_feature['x_059/x_55'] = data['x_059'] / (data['x_055'] + 1e-10) 
    model_sample_strong_feature['x_064/x_062'] = data['x_064'] / (data['x_062'] + 1e-10)
    model_sample_strong_feature['x_067/x_065'] = data['x_067'] / (data['x_065'] + 1e-10)  
    model_sample_strong_feature['x_070/x_068'] = data['x_070'] / (data['x_068'] + 1e-10)
    model_sample_strong_feature['x_073/x_071'] = data['x_073'] / (data['x_071'] + 1e-10) 
    model_sample_strong_feature['x_078/x_074'] = data['x_078'] / (data['x_074'] + 1e-10)  
    model_sample_strong_feature['x_085/x_081'] = data['x_085'] / (data['x_081'] + 1e-10) 
    model_sample_strong_feature['x_100/x_101'] = data['x_100'] / (data['x_101'] + 1e-10)
    model_sample_strong_feature['x_102/x_103'] = data['x_102'] / (data['x_103'] + 1e-10) 
    model_sample_strong_feature['x_108/x_105'] = data['x_108'] / (data['x_105'] + 1e-10)
    model_sample_strong_feature['x_104/x_102'] = data['x_104'] / (data['x_102'] + 1e-10) 
    model_sample_strong_feature['x_109/x_110'] = data['x_109'] / (data['x_110'] + 1e-10)
    model_sample_strong_feature['x_111/x_109'] = data['x_111'] / (data['x_109'] + 1e-10) 
    model_sample_strong_feature['x_112/x_113'] = data['x_112'] / (data['x_113'] + 1e-10)
    model_sample_strong_feature['x_114/x_112'] = data['x_114'] / (data['x_112'] + 1e-10) 
    model_sample_strong_feature['x_115/x_116'] = data['x_115'] / (data['x_116'] + 1e-10)
    model_sample_strong_feature['x_117/x_115'] = data['x_117'] / (data['x_115'] + 1e-10)  
    model_sample_strong_feature['x_118/x_119'] = data['x_118'] / (data['x_119'] + 1e-10)
    model_sample_strong_feature['x_120/x_118'] = data['x_120'] / (data['x_118'] + 1e-10) 
    model_sample_strong_feature['x_125/x_121'] = data['x_125'] / (data['x_121'] + 1e-10) 
    model_sample_strong_feature['x_128/x_129'] = data['x_128'] / (data['x_129'] + 1e-10)
    model_sample_strong_feature['x_130/x_128'] = data['x_130'] / (data['x_128'] + 1e-10)

    # 每笔放款金额，每个机构的放款笔数，每个机构的放款金额
    model_sample_strong_feature['x_133/x_134'] = data['x_133'] / (data['x_134'] + 1e-10)
    model_sample_strong_feature['x_133/x_132'] = data['x_133'] / (data['x_132'] + 1e-10)
    model_sample_strong_feature['x_134/x_132'] = data['x_134'] / (data['x_132'] + 1e-10) 
    model_sample_strong_feature['x_138/x_139'] = data['x_138'] / (data['x_139'] + 1e-10)
    model_sample_strong_feature['x_138/x_137'] = data['x_138'] / (data['x_137'] + 1e-10)
    model_sample_strong_feature['x_139/x_137'] = data['x_139'] / (data['x_137'] + 1e-10) 
    model_sample_strong_feature['x_143/x_142'] = data['x_143'] / (data['x_142'] + 1e-10)
    model_sample_strong_feature['x_143/x_144'] = data['x_143'] / (data['x_144'] + 1e-10)
    model_sample_strong_feature['x_144/x_142'] = data['x_144'] / (data['x_142'] + 1e-10)

    # 每个机构的放款均值,失败还款笔数占比
    model_sample_strong_feature['x_151/x_149'] = data['x_151'] / (data['x_149'] + 1e-10)
    model_sample_strong_feature['x_152/x_149'] = data['x_152'] / (data['x_149'] + 1e-10)
    model_sample_strong_feature['x_152/x_151'] = data['x_152'] / (data['x_151'] + 1e-10)
    model_sample_strong_feature['x_154/x_153'] = data['x_154'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_156/x_153'] = data['x_156'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_157/x_153'] = data['x_157'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_158/x_153'] = data['x_158'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_159/x_153'] = data['x_159'] / (data['x_153'] + 1e-10)  
    model_sample_strong_feature['x_154/x_155'] = data['x_154'] / (data['x_155'] + 1e-10)  

    model_sample_strong_feature['x_164/x_162'] = data['x_164'] / (data['x_162'] + 1e-10)
    model_sample_strong_feature['x_165/x_162'] = data['x_165'] / (data['x_162'] + 1e-10)
    model_sample_strong_feature['x_165/x_164'] = data['x_165'] / (data['x_164'] + 1e-10)
    model_sample_strong_feature['x_167/x_166'] = data['x_167'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_169/x_166'] = data['x_169'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_170/x_166'] = data['x_170'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_171/x_166'] = data['x_171'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_180/x_181'] = data['x_180'] / (data['x_181'] + 1e-10) 
    model_sample_strong_feature['x_167/x_168'] = data['x_167'] / (data['x_168'] + 1e-10) 
    model_sample_strong_feature['x_172/x_167'] = data['x_172'] / (data['x_167'] + 1e-10)  

    model_sample_strong_feature['x_177/x_175'] = data['x_177'] / (data['x_175'] + 1e-10)
    model_sample_strong_feature['x_178/x_175'] = data['x_178'] / (data['x_175'] + 1e-10)
    model_sample_strong_feature['x_178/x_177'] = data['x_178'] / (data['x_177'] + 1e-10)
    model_sample_strong_feature['x_180/x_179'] = data['x_180'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_182/x_179'] = data['x_182'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_183/x_179'] = data['x_183'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_184/x_179'] = data['x_184'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_180/x_181'] = data['x_180'] / (data['x_181'] + 1e-10) 
    model_sample_strong_feature['x_185/x_180'] = data['x_185'] / (data['x_180'] + 1e-10)
 
    # 90天与30天的申请贷款机构的趋势，180天与90天的申请贷款机构的趋势，180天与30天的申请贷款机构的趋势；90天与30天的成功申请贷款机构的趋势，180天与90天的成功申请贷款机构的趋势，180天；
    # 30天的成功申请贷款机构的趋势；90天与30天的申请贷款笔数的趋势，180天与90天的申请贷款笔数的趋势，180天与30天的申请贷款笔数的趋势90天的申请贷款笔数的趋势
    model_sample_strong_feature['x_189/x_188'] = data['x_189'] / (data['x_188'] + 1e-10)
    model_sample_strong_feature['x_191/x_190'] = data['x_191'] / (data['x_190'] + 1e-10)
    model_sample_strong_feature['x_193/x_192'] = data['x_193'] / (data['x_192'] + 1e-10)
    model_sample_strong_feature['x_195/x_194'] = data['x_195'] / (data['x_194'] + 1e-10)
    model_sample_strong_feature['x_197/x_196'] = data['x_197'] / (data['x_196'] + 1e-10)
    model_sample_strong_feature['x_199/x_198'] = data['x_199'] / (data['x_198'] + 1e-10)
    model_sample_strong_feature['x_196/x_188'] = data['x_196'] / (data['x_188'] + 1e-10)
    model_sample_strong_feature['x_192/x_188'] = data['x_192'] / (data['x_188'] + 1e-10)
                                                                   
    model_sample_strong_feature = model_sample_strong_feature.fillna(-999)
    return model_sample_strong_feature 

In [25]:
get_features_middle(model_sample).descibe()

AttributeError: 'DataFrame' object has no attribute 'descibe'

In [9]:
def get_score(y_pred, y_true):
    acc_ = accuracy_score(y_true=y_true,y_pred=y_pred)
    TP = np.sum(((y_pred == 1) & (y_true == 1))) 
    precision = TP / np.sum(y_pred)
    recall = TP / np.sum(y_true)
    print('TP: ',TP,'/', np.sum(y_true), 'all ',np.sum(y_pred), ' accuracy: ',acc_, ' precision: ',precision, ' recall: ',recall, ' F_score: ', 2 * precision * recall / (precision + recall),fbeta_score(y_true=y_true,y_pred=y_pred,beta=1) )
    

In [10]:
def N_Fold_Predict( train_fea , train_y,  test_fea, cv_ = 5):

    ###########################################################
    train_fea = train_fea.fillna(-1)
    test_fea = test_fea.fillna(-1)

    features_col = [c for c in train_fea.columns if c not in ['user_id','y']]
    X = train_fea[features_col] 
    X_pred = test_fea[features_col]
    
    pred_out_lgb = 0
    pred_out_gbdt = 0
    pred_out_rf = 0
    
    for cv in range(cv_):
        X_train, X_test, y_train, y_test = train_test_split(X, train_y, test_size=0.25, random_state=np.random.randint(1000)) 
        # create dataset for lightgbm
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

        # specify your configurations as a dict
        params = {
            'task':'train',
            'boosting_type':'gbdt',
            'num_leaves': 31,
            'objective': 'binary', 
            'learning_rate': 0.05, 
            'bagging_freq': 2, 
            'max_bin':256,
            'num_threads': 32
        } 

        # train
        gbm = lgb.train(params,
                    lgb_train,
                    verbose_eval= 0,
                    num_boost_round=10000,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=100)

        lgb_pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)
        
        
        gbdt = GradientBoostingClassifier(n_estimators=250,learning_rate=0.01,max_depth=6,min_samples_leaf=5,min_samples_split=5)
        gbdt.fit(X_train, y_train)
        gbdt_pred = gbdt.predict_proba(X_pred)[:,1] 
        
        
        rf = RandomForestClassifier(n_estimators=500,max_depth=6,min_samples_leaf=5,min_samples_split=5)
        rf.fit(X_train, y_train)
        rf_pred = rf.predict_proba(X_pred)[:,1]  
        
        if cv == 0:
            pred_out_lgb = lgb_pred
            pred_out_gbdt = gbdt_pred
            pred_out_rf = rf_pred
        else:
            pred_out_lgb += lgb_pred
            pred_out_gbdt += gbdt_pred
            pred_out_rf += rf_pred
            
    pred_out_lgb = pred_out_lgb * 1.0 / cv_
    pred_out_gbdt = pred_out_gbdt * 1.0 / cv_
    pred_out_rf = pred_out_rf * 1.0 / cv_
    return pred_out_lgb, pred_out_gbdt, pred_out_rf

In [14]:
def get_features_final(data):
    model_sample_strong_feature = data.copy()
    
    first_strong_features = ['x_003','x_004','x_005','x_006','x_007','x_008','x_009','x_010','x_011','x_012','x_013','x_014','x_015','x_016','x_017','x_018','x_019']

    res = 0
    for i in range(len(first_strong_features)):
        res += 2 ** i * data[first_strong_features[i]] 
    model_sample_strong_feature['x_1_strong'] = res

    model_sample_strong_feature['x_022/x_020'] = data['x_022'] / (data['x_020'] + 1e-10)
    model_sample_strong_feature['x_023/x_020'] = data['x_023'] / (data['x_020'] + 1e-10)
    model_sample_strong_feature['x_024/x_020'] = data['x_024'] /  (data['x_020'] + 1e-10)
    model_sample_strong_feature['x_025/x_020'] = data['x_025'] /  (data['x_020'] + 1e-10)
    model_sample_strong_feature['x_026/x_020'] = data['x_026'] /  (data['x_020'] + 1e-10)
    
    
    model_sample_strong_feature['x_028/x_021'] = data['x_028'] / (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_029/x_021'] = data['x_029'] / (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_030/x_021'] = data['x_030'] /  (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_031/x_021'] = data['x_031'] /  (data['x_021'] + 1e-10)
    model_sample_strong_feature['x_032/x_021'] = data['x_032'] /  (data['x_021'] + 1e-10)
    
    
    model_sample_strong_feature['all_cards'] = (data['x_034']  + data['x_035'] + data['x_036'] + data['x_037'] + data['x_038'] + data['x_039'] + data['x_040']).values

    model_sample_strong_feature['x_034/all_cards'] = data['x_034'] / (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_035/all_cards'] = data['x_035'] / (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_036/all_cards'] = data['x_036'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_037/all_cards'] = data['x_037'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_038/all_cards'] = data['x_038'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_039/all_cards'] = data['x_039'] /  (model_sample_strong_feature['all_cards'] + 1e-10)
    model_sample_strong_feature['x_040/all_cards'] = data['x_040'] /  (model_sample_strong_feature['all_cards'] + 1e-10)

    model_sample_strong_feature['x_027/x_033'] = data['x_027'] -  (data['x_033'] + 1e-10)
    
    
    
    model_sample_strong_feature['x_043/x_044'] = data['x_043'] / (data['x_044'] + 1e-10)
    model_sample_strong_feature['x_046/x_047'] = data['x_046'] / (data['x_047'] + 1e-10)
    model_sample_strong_feature['x_050/x_051'] = data['x_050'] / (data['x_051'] + 1e-10)
    model_sample_strong_feature['x_053/x_054'] = data['x_053'] / (data['x_054'] + 1e-10)
    model_sample_strong_feature['x_057/x_058'] = data['x_057'] / (data['x_058'] + 1e-10)
    model_sample_strong_feature['x_060/x_061'] = data['x_060'] / (data['x_061'] + 1e-10)
    model_sample_strong_feature['x_076/x_077'] = data['x_076'] / (data['x_077'] + 1e-10)
    model_sample_strong_feature['x_079/x_080'] = data['x_079'] / (data['x_080'] + 1e-10)
    model_sample_strong_feature['x_083/x_084'] = data['x_083'] / (data['x_084'] + 1e-10) 
    model_sample_strong_feature['x_086/x_087'] = data['x_086'] / (data['x_087'] + 1e-10)
    model_sample_strong_feature['x_090/x_091'] = data['x_090'] / (data['x_091'] + 1e-10)
    model_sample_strong_feature['x_094/x_095'] = data['x_094'] / (data['x_095'] + 1e-10)
    model_sample_strong_feature['x_098/x_099'] = data['x_098'] / (data['x_099'] + 1e-10) 
    model_sample_strong_feature['x_123/x_124'] = data['x_123'] / (data['x_124'] + 1e-10)
    model_sample_strong_feature['x_126/x_127'] = data['x_126'] / (data['x_127'] + 1e-10) 
    
    
    model_sample_strong_feature['x_064/x_063'] = data['x_064'] / (data['x_063'] + 1e-10)
    model_sample_strong_feature['x_067/x_066'] = data['x_067'] / (data['x_066'] + 1e-10)
    model_sample_strong_feature['x_070/x_069'] = data['x_070'] / (data['x_069'] + 1e-10)
    model_sample_strong_feature['x_073/x_072'] = data['x_073'] / (data['x_072'] + 1e-10)
    
    model_sample_strong_feature['x_059/x_55'] = data['x_059'] / (data['x_055'] + 1e-10)
    model_sample_strong_feature['x_067/x_065'] = data['x_067'] / (data['x_065'] + 1e-10)
    model_sample_strong_feature['x_070/x_068'] = data['x_070'] / (data['x_068'] + 1e-10)
    model_sample_strong_feature['x_073/x_071'] = data['x_073'] / (data['x_071'] + 1e-10)
    model_sample_strong_feature['x_078/x_074'] = data['x_078'] / (data['x_074'] + 1e-10)
    model_sample_strong_feature['x_085/x_081'] = data['x_085'] / (data['x_081'] + 1e-10)
    model_sample_strong_feature['x_100/x_101'] = data['x_100'] / (data['x_101'] + 1e-10)
    model_sample_strong_feature['x_102/x_103'] = data['x_102'] / (data['x_103'] + 1e-10)
    model_sample_strong_feature['x_108/x_105'] = data['x_108'] / (data['x_105'] + 1e-10)
    model_sample_strong_feature['x_104/x_102'] = data['x_104'] / (data['x_102'] + 1e-10)
    model_sample_strong_feature['x_109/x_110'] = data['x_109'] / (data['x_110'] + 1e-10)
    model_sample_strong_feature['x_111/x_109'] = data['x_111'] / (data['x_109'] + 1e-10)
    model_sample_strong_feature['x_112/x_113'] = data['x_112'] / (data['x_113'] + 1e-10)
    model_sample_strong_feature['x_114/x_112'] = data['x_114'] / (data['x_112'] + 1e-10)
    model_sample_strong_feature['x_115/x_116'] = data['x_115'] / (data['x_116'] + 1e-10)
    model_sample_strong_feature['x_117/x_115'] = data['x_117'] / (data['x_115'] + 1e-10) 
    model_sample_strong_feature['x_118/x_119'] = data['x_118'] / (data['x_119'] + 1e-10)
    model_sample_strong_feature['x_120/x_118'] = data['x_120'] / (data['x_118'] + 1e-10) 
    model_sample_strong_feature['x_125/x_121'] = data['x_125'] / (data['x_121'] + 1e-10) 
    model_sample_strong_feature['x_128/x_129'] = data['x_128'] / (data['x_129'] + 1e-10)
    model_sample_strong_feature['x_130/x_128'] = data['x_130'] / (data['x_128'] + 1e-10) 
    
    
    model_sample_strong_feature['x_133/x_134'] = data['x_133'] / (data['x_134'] + 1e-10)
    model_sample_strong_feature['x_133/x_132'] = data['x_133'] / (data['x_132'] + 1e-10)
    model_sample_strong_feature['x_134/x_132'] = data['x_134'] / (data['x_132'] + 1e-10)

    model_sample_strong_feature['x_138/x_139'] = data['x_138'] / (data['x_139'] + 1e-10)
    model_sample_strong_feature['x_138/x_137'] = data['x_138'] / (data['x_137'] + 1e-10)
    model_sample_strong_feature['x_139/x_137'] = data['x_139'] / (data['x_137'] + 1e-10)

    model_sample_strong_feature['x_143/x_142'] = data['x_143'] / (data['x_142'] + 1e-10)
    model_sample_strong_feature['x_143/x_144'] = data['x_143'] / (data['x_144'] + 1e-10)
    model_sample_strong_feature['x_144/x_142'] = data['x_144'] / (data['x_142'] + 1e-10)

    model_sample_strong_feature['x_151/x_149'] = data['x_151'] / (data['x_149'] + 1e-10)
    model_sample_strong_feature['x_152/x_149'] = data['x_152'] / (data['x_149'] + 1e-10)
    model_sample_strong_feature['x_152/x_151'] = data['x_152'] / (data['x_151'] + 1e-10)
    model_sample_strong_feature['x_154/x_153'] = data['x_154'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_156/x_153'] = data['x_156'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_157/x_153'] = data['x_157'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_158/x_153'] = data['x_158'] / (data['x_153'] + 1e-10)
    model_sample_strong_feature['x_159/x_153'] = data['x_159'] / (data['x_153'] + 1e-10)  
    model_sample_strong_feature['x_154/x_155'] = data['x_154'] / (data['x_155'] + 1e-10)  


    model_sample_strong_feature['x_164/x_162'] = data['x_164'] / (data['x_162'] + 1e-10)
    model_sample_strong_feature['x_165/x_162'] = data['x_165'] / (data['x_162'] + 1e-10)
    model_sample_strong_feature['x_165/x_164'] = data['x_165'] / (data['x_164'] + 1e-10)
    model_sample_strong_feature['x_167/x_166'] = data['x_167'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_169/x_166'] = data['x_169'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_170/x_166'] = data['x_170'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_171/x_166'] = data['x_171'] / (data['x_166'] + 1e-10)
    model_sample_strong_feature['x_180/x_181'] = data['x_180'] / (data['x_181'] + 1e-10) 
    model_sample_strong_feature['x_167/x_168'] = data['x_167'] / (data['x_168'] + 1e-10) 
    model_sample_strong_feature['x_172/x_167'] = data['x_172'] / (data['x_167'] + 1e-10)  
    
    model_sample_strong_feature['x_177/x_175'] = data['x_177'] / (data['x_175'] + 1e-10)
    model_sample_strong_feature['x_178/x_175'] = data['x_178'] / (data['x_175'] + 1e-10)
    model_sample_strong_feature['x_178/x_177'] = data['x_178'] / (data['x_177'] + 1e-10)
    model_sample_strong_feature['x_180/x_179'] = data['x_180'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_182/x_179'] = data['x_182'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_183/x_179'] = data['x_183'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_184/x_179'] = data['x_184'] / (data['x_179'] + 1e-10)
    model_sample_strong_feature['x_180/x_181'] = data['x_180'] / (data['x_181'] + 1e-10) 
    model_sample_strong_feature['x_185/x_180'] = data['x_185'] / (data['x_180'] + 1e-10)
 
    model_sample_strong_feature['x_189/x_188'] = data['x_189'] / (data['x_188'] + 1e-10)
    model_sample_strong_feature['x_191/x_190'] = data['x_191'] / (data['x_190'] + 1e-10)
    model_sample_strong_feature['x_193/x_192'] = data['x_193'] / (data['x_192'] + 1e-10)
    model_sample_strong_feature['x_195/x_194'] = data['x_195'] / (data['x_194'] + 1e-10)
    model_sample_strong_feature['x_197/x_196'] = data['x_197'] / (data['x_196'] + 1e-10)
    model_sample_strong_feature['x_199/x_198'] = data['x_199'] / (data['x_198'] + 1e-10)
    model_sample_strong_feature['x_196/x_188'] = data['x_196'] / (data['x_188'] + 1e-10)
    model_sample_strong_feature['x_192/x_188'] = data['x_192'] / (data['x_188'] + 1e-10)

    
    model_sample_strong_feature = model_sample_strong_feature.fillna(-999)
    return model_sample_strong_feature 

In [15]:
model_sample_strong_feature_middle = get_features_middle(model_sample)
model_sample_strong_feature_final = get_features_final(model_sample)
model_sample_strong_feature_middle = model_sample_strong_feature_middle.fillna(-999)
model_sample_strong_feature_final = model_sample_strong_feature_final.fillna(-999)
model_sample_ = model_sample.fillna(-999)

In [24]:
for rnd in [1,10,100,1000]:
    print('Random Seed is: ',rnd)  
    train_X,test_X, train_y, test_y = train_test_split(model_sample_strong_feature_final,label,test_size=0.2,random_state=rnd) 
    
    train_X_orig = model_sample_.loc[train_X.index]
    test_X_orig = model_sample_.loc[test_X.index]
    train_X_middle = model_sample_strong_feature_middle.loc[train_X.index]
    test_X_middle = model_sample_strong_feature_middle.loc[test_X.index]
    
    
    print('5 fold no feature engineering')
    pred_out_lgb, pred_out_gbdt, pred_out_rf = N_Fold_Predict(train_X_orig, train_y['y'].values, test_X_orig, cv_ = 3)
    pred =pred_out_rf >= 0.215
    get_score(pred, test_y['y'].values)
    pred =pred_out_gbdt >= 0.215
    get_score(pred, test_y['y'].values)
    pred =pred_out_lgb >= 0.215
    get_score(pred, test_y['y'].values)
    pred =pred_out_lgb * 0.55 + 0.45 * pred_out_gbdt>= 0.215
    get_score(pred, test_y['y'].values)
    pred =pred_out_lgb * 0.5 + 0.5 * pred_out_gbdt>= 0.215
    get_score(pred, test_y['y'].values)
    pred =(pred_out_lgb * 0.5 + 0.5 * pred_out_gbdt) * 0.9 + 0.1 * pred_out_rf>= 0.215
    get_score(pred, test_y['y'].values)
    
    print('*' * 50)
    print('5 fold feature engineering middle')
    pred_out_lgb_middle, pred_out_gbdt_middle, pred_out_rf_middle = N_Fold_Predict(train_X_middle,train_y['y'].values, test_X_middle, cv_ = 3)
    pred = pred_out_rf_middle >= 0.215
    get_score(pred, test_y['y'].values)
    pred =pred_out_gbdt_middle >= 0.215
    get_score(pred, test_y['y'].values)
    pred =pred_out_lgb_middle >= 0.215
    get_score(pred, test_y['y'].values)
    
    pred =pred_out_lgb_middle * 0.55 + 0.45 * pred_out_gbdt_middle>= 0.215
    get_score(pred, test_y['y'].values)
    pred =pred_out_lgb_middle * 0.5 + 0.5 * pred_out_gbdt_middle>= 0.215
    get_score(pred, test_y['y'].values)
    pred =(pred_out_lgb_middle * 0.5 + 0.5 * pred_out_gbdt_middle) * 0.9 + 0.1 * pred_out_rf_middle >= 0.215
    get_score(pred, test_y['y'].values)
     
#     pred =(pred_out_lgb_middle * 0.5 + 0.5 * pred_out_gbdt_middle) * 0.9 + 0.05 * (pred_out_rf_middle + pred_out_rf)>= 0.215
#     get_score(pred, test_y['y'].values)
     
    
    print('*' * 50)
    print('5 fold feature engineering final')
    pred_out_lgb_final, pred_out_gbdt_final, pred_out_rf_final = N_Fold_Predict(train_X,train_y['y'].values, test_X, cv_ = 3)
    pred =pred_out_rf_final >= 0.215
    get_score(pred, test_y['y'].values)
    pred =pred_out_gbdt_final >= 0.215
    get_score(pred, test_y['y'].values)
    pred =pred_out_lgb_final >= 0.215
    get_score(pred, test_y['y'].values)
    
    pred =pred_out_lgb_final * 0.55 + 0.45 * pred_out_gbdt_final>= 0.215
    get_score(pred, test_y['y'].values)
    pred =pred_out_lgb_final * 0.5 + 0.5 * pred_out_gbdt_final>= 0.215
    get_score(pred, test_y['y'].values)
    pred =(pred_out_lgb_final * 0.5 + 0.5 * pred_out_gbdt_final) * 0.9 + 0.1 * pred_out_rf_final>= 0.215
    get_score(pred, test_y['y'].values)
    
    print('*' * 50)
     
    print('Fire!')
    print('middle  and  original ')
    
    pred =((pred_out_lgb_middle * 0.5 + pred_out_lgb * 0.5 )*0.55  + 0.45 * (pred_out_gbdt_middle * 0.5 + 0.5 * pred_out_gbdt))>= 0.215
    get_score(pred, test_y['y'].values)
    pred =((pred_out_lgb_middle * 0.5 + pred_out_lgb* 0.5 )*0.5  + 0.5 * (pred_out_gbdt_middle *0.5 + 0.5 * pred_out_gbdt ))>= 0.215
    get_score(pred, test_y['y'].values) 
     
    print('final and  original ')
    pred =((pred_out_lgb_final * 0.5 + pred_out_lgb * 0.5 )*0.55  + 0.45 * (pred_out_gbdt_final * 0.5 + 0.5 * pred_out_gbdt))>= 0.215
    get_score(pred, test_y['y'].values)
    pred =((pred_out_lgb_final * 0.5 + pred_out_lgb* 0.5 )*0.5  + 0.5 * (pred_out_gbdt_final *0.5 + 0.5 * pred_out_gbdt ))>= 0.215
    get_score(pred, test_y['y'].values) 
     
    
    print('final and  middle and original ')
    pred =((pred_out_lgb_final * 0.3 + pred_out_lgb * 0.3 + 0.4 * pred_out_lgb_middle)*0.55  + 0.45 * (pred_out_gbdt_final * 0.3 + 0.3 * pred_out_gbdt + 0.4 * pred_out_gbdt_middle))>= 0.215
    get_score(pred, test_y['y'].values)
    pred =((pred_out_lgb_final * 1.0 /3 + pred_out_lgb* 1.0 /3 +  pred_out_lgb_middle* 1.0 /3)*0.5  + 0.5 * (pred_out_gbdt_final * 1.0 /3 +  1.0 /3 * pred_out_gbdt + 1.0 /3 * pred_out_gbdt_middle))>= 0.215
    get_score(pred, test_y['y'].values) 
    
    pred = 0.1 * (pred_out_rf_middle + pred_out_rf + pred_out_rf_final ) /3.0 + 0.9 * ((pred_out_lgb_final * 1.0 /3 + pred_out_lgb* 1.0 /3 +  pred_out_lgb_middle* 1.0 /3)*0.5  + 0.5 * (pred_out_gbdt_final * 1.0 /3 +  1.0 /3 * pred_out_gbdt + 1.0 /3 * pred_out_gbdt_middle))>= 0.215
    get_score(pred, test_y['y'].values) 
    pred = 0.15 * (pred_out_rf_middle + pred_out_rf + pred_out_rf_final ) /3.0 + 0.85 * ((pred_out_lgb_final * 1.0 /3 + pred_out_lgb* 1.0 /3 +  pred_out_lgb_middle* 1.0 /3)*0.5  + 0.5 * (pred_out_gbdt_final * 1.0 /3 +  1.0 /3 * pred_out_gbdt + 1.0 /3 * pred_out_gbdt_middle))>= 0.215
    get_score(pred, test_y['y'].values) 

Random Seed is:  1
5 fold no feature engineering
TP:  312 / 390 all  870  accuracy:  0.6826347305389222  precision:  0.3586206896551724  recall:  0.8  F_score:  0.49523809523809514 0.49523809523809514
TP:  288 / 390 all  732  accuracy:  0.7275449101796407  precision:  0.39344262295081966  recall:  0.7384615384615385  F_score:  0.5133689839572192 0.5133689839572192
TP:  270 / 390 all  719  accuracy:  0.7160678642714571  precision:  0.37552155771905427  recall:  0.6923076923076923  F_score:  0.48692515779981965 0.48692515779981965
TP:  277 / 390 all  739  accuracy:  0.7130738522954092  precision:  0.37483085250338294  recall:  0.7102564102564103  F_score:  0.4906997342781223 0.4906997342781223
TP:  277 / 390 all  739  accuracy:  0.7130738522954092  precision:  0.37483085250338294  recall:  0.7102564102564103  F_score:  0.4906997342781223 0.4906997342781223
TP:  284 / 390 all  749  accuracy:  0.7150698602794411  precision:  0.37917222963951935  recall:  0.7282051282051282  F_score:  0.498

5 fold no feature engineering
TP:  281 / 386 all  814  accuracy:  0.6816367265469062  precision:  0.3452088452088452  recall:  0.727979274611399  F_score:  0.4683333333333333 0.4683333333333333
TP:  262 / 386 all  692  accuracy:  0.7235528942115769  precision:  0.3786127167630058  recall:  0.6787564766839378  F_score:  0.48608534322820035 0.48608534322820035
TP:  261 / 386 all  688  accuracy:  0.7245508982035929  precision:  0.3793604651162791  recall:  0.6761658031088082  F_score:  0.48603351955307256 0.48603351955307256
TP:  265 / 386 all  690  accuracy:  0.7275449101796407  precision:  0.38405797101449274  recall:  0.6865284974093264  F_score:  0.4925650557620818 0.4925650557620818
TP:  264 / 386 all  687  accuracy:  0.7280439121756487  precision:  0.38427947598253276  recall:  0.6839378238341969  F_score:  0.49207828518173347 0.49207828518173347
TP:  264 / 386 all  700  accuracy:  0.7215568862275449  precision:  0.37714285714285717  recall:  0.6839378238341969  F_score:  0.48618784